<a href="https://colab.research.google.com/github/Lima-Ricardo/Analise-Pandas-Pyspark-GCP/blob/main/Analise_de_dados_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import datetime
import os
from google.cloud import storage

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#thousands = '.', decimal = ',', dtype = {'dotacao':np.float64,'empenhada': np.float64})
df_A = (pd.read_csv('/content/drive/MyDrive/arquivos pipeline/producao-mar-2016-2018.csv', sep=',',encoding="ISO-8859-1", thousands = '.', decimal = ',', dtype = {'Produção de Óleo (m³)':np.float64,'Produção de Condensado (m³)': np.float64, 'Produção de Gás Associado (Mm³)': np.float64, 'Produção de Gás Não Associado (Mm³)': np.float64, "Produção de Água (m³)":np.float64, 'Injeção de Gás (Mm³)':np.float64 }))
df_B = (pd.read_csv('/content/drive/MyDrive/arquivos pipeline/producao-mar-2019.csv', sep=';',encoding="ISO-8859-1", thousands = '.', decimal = ',', dtype = {'Produção de Óleo (m³)':np.float64,'Produção de Condensado (m³)': np.float64, 'Produção de Gás Associado (Mm³)': np.float64, 'Produção de Gás Não Associado (Mm³)': np.float64, "Produção de Água (m³)":np.float64, 'Injeção de Gás (Mm³)':np.float64 }))
df_C = (pd.read_csv('/content/drive/MyDrive/arquivos pipeline/producao-mar-2020.csv', sep=';',encoding="utf-8", thousands = '.', decimal = ',', dtype = {'Produção de Óleo (m³)':np.float64,'Produção de Condensado (m³)': np.float64, 'Produção de Gás Associado (Mm³)': np.float64, 'Produção de Gás Não Associado (Mm³)': np.float64, "Produção de Água (m³)":np.float64, 'Injeção de Gás (Mm³)':np.float64 }))
df_D = (pd.read_csv('/content/drive/MyDrive/arquivos pipeline/producao-mar-2021.csv', sep=';',encoding="ISO-8859-1", thousands = '.', decimal = ',', dtype = {'Produção de Óleo (m³)':np.float64,'Produção de Condensado (m³)': np.float64, 'Produção de Gás Associado (Mm³)': np.float64, 'Produção de Gás Não Associado (Mm³)': np.float64, "Produção de Água (m³)":np.float64, 'Injeção de Gás (Mm³)':np.float64 }))
df_E = (pd.read_csv('/content/drive/MyDrive/arquivos pipeline/tratado-producao-mar-2016-2018.csv', sep=',',encoding="utf-8", thousands = '.', decimal = ',', dtype = {'Produção de Óleo (m³)':np.float64,'Produção de Condensado (m³)': np.float64, 'Produção de Gás Associado (Mm³)': np.float64, 'Produção de Gás Não Associado (Mm³)': np.float64, "Produção de Água (m³)":np.float64, 'Injeção de Gás (Mm³)':np.float64 }))


In [ ]:
display(df_D)

,"Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)"
0,"2021,01/2021,Alagoas,Alagoas,PARU,4-ALS-39-AL,..."
1,"2021,01/2021,Bahia,Camamu,MANATI,7-MNT-1-BAS,M..."
2,"2021,01/2021,Bahia,Camamu,MANATI,7-MNT-2-BAS,M..."
3,"2021,01/2021,Bahia,Camamu,MANATI,7-MNT-3-BAS,M..."
4,"2021,01/2021,Bahia,Camamu,MANATI,7-MNT-4-BAS,M..."
...,...
21144,"2021,12/2021,São Paulo,Santos,SUDOESTE DE SAPI..."
21145,"2021,12/2021,São Paulo,Santos,SUDOESTE DE SAPI..."
21146,"2021,12/2021,São Paulo,Santos,SUDOESTE DE SAPI..."
21147,"2021,12/2021,São Paulo,Santos,SUDOESTE DE SAPI..."


In [ ]:
def dados_perdidos(df_B):
    perdas = df_B.isna().sum()
    perdas = perdas[perdas > 0 ]
    perdas_proporcionais = perdas/df_B.shape[0]
    perdas_totais = perdas_proporcionais > 0.05

    return pd.DataFrame({"Perdas" : perdas, "Proporção de perdas" : perdas_proporcionais, "Perdas maiores que 5%" : perdas_totais})


dados_perdidos(df_B)

,Perdas,Proporção de perdas,Perdas maiores que 5%


Começando o tratamento e unificando todos os datasets

In [ ]:
df1 = df_A.copy()
df2 = df_B.copy()
df3 = df_C.copy()
df4 = df_D.copy()
df5 = df_E.copy()

In [ ]:
display(df3)

,"Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)"
0,"2020,01/2020,Alagoas,Alagoas,PARU,4-ALS-39-AL,..."
1,"2020,01/2020,Bahia,Camamu,MANATI,7-MNT-1-BAS,M..."
2,"2020,01/2020,Bahia,Camamu,MANATI,7-MNT-2-BAS,M..."
3,"2020,01/2020,Bahia,Camamu,MANATI,7-MNT-3-BAS,M..."
4,"2020,01/2020,Bahia,Camamu,MANATI,7-MNT-4-BAS,M..."
...,...
21810,"2020,12/2020,São Paulo,Santos,SUDOESTE DE SAPI..."
21811,"2020,12/2020,São Paulo,Santos,SUDOESTE DE SAPI..."
21812,"2020,12/2020,São Paulo,Santos,SUDOESTE DE SAPI..."
21813,"2020,12/2020,São Paulo,Santos,SUDOESTE DE SAPI..."


In [ ]:
#Criando um novo dataframe corrigindo o problema do csv original apenas se o separador for ponto e vírgula(;)
df2['Ano'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(0)
df2['Mês'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(1)
df2['Estado'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(2)
df2['Bacia'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(3)
df2['Campo'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(4)
df2['Poço'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(5)
df2['Ambiente'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(6)
df2['Instalação'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(7)
df2['Produção de Óleo (m³)'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(8)
df2['Produção de Condensado (m³)'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(9)
df2['Produção de Gás Associado (Mm³)'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(10)
df2['Produção de Gás Não Associado (Mm³)'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(11)
df2['Produção de Água (m³)'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(12)
df2['Injeção de Gás (Mm³)'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(13)
df2['Injeção de Água para Recuperação Secundária (m³)'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(14)
df2['Injeção de Água para Descarte (m³)'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(15)
df2['Injeção de Gás Carbônico (Mm³)'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(16)
df2['Injeção de Nitrogênio (Mm³)'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(17)
df2['Injeção de Vapor de Água (t)'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(18)
df2['Injeção de Polímeros (m³)'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(19)
df2['Injeção de Outros Fluidos (m³)'] = df2['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(20)



In [ ]:
#Criando um novo dataframe corrigindo o problema do csv original apenas se o separador for ponto e vírgula(;)
df3['Ano'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(0)
df3['Mês'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(1)
df3['Estado'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(2)
df3['Bacia'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(3)
df3['Campo'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(4)
df3['Poço'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(5)
df3['Ambiente'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(6)
df3['Instalação'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(7)
df3['Produção de Óleo (m³)'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(8)
df3['Produção de Condensado (m³)'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(9)
df3['Produção de Gás Associado (Mm³)'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(10)
df3['Produção de Gás Não Associado (Mm³)'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(11)
df3['Produção de Água (m³)'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(12)
df3['Injeção de Gás (Mm³)'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(13)
df3['Injeção de Água para Recuperação Secundária (m³)'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(14)
df3['Injeção de Água para Descarte (m³)'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(15)
df3['Injeção de Gás Carbônico (Mm³)'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(16)
df3['Injeção de Nitrogênio (Mm³)'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(17)
df3['Injeção de Vapor de Água (t)'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(18)
df3['Injeção de Polímeros (m³)'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(19)
df3['Injeção de Outros Fluidos (m³)'] = df3['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(20)



In [ ]:
#Criando um novo dataframe corrigindo o problema do csv original apenas se o separador for ponto e vírgula(;)
df4['Ano'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(0)
df4['Mês'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(1)
df4['Estado'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(2)
df4['Bacia'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(3)
df4['Campo'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(4)
df4['Poço'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(5)
df4['Ambiente'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(6)
df4['Instalação'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(7)
df4['Produção de Óleo (m³)'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(8)
df4['Produção de Condensado (m³)'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(9)
df4['Produção de Gás Associado (Mm³)'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(10)
df4['Produção de Gás Não Associado (Mm³)'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(11)
df4['Produção de Água (m³)'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(12)
df4['Injeção de Gás (Mm³)'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(13)
df4['Injeção de Água para Recuperação Secundária (m³)'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(14)
df4['Injeção de Água para Descarte (m³)'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(15)
df4['Injeção de Gás Carbônico (Mm³)'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(16)
df4['Injeção de Nitrogênio (Mm³)'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(17)
df4['Injeção de Vapor de Água (t)'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(18)
df4['Injeção de Polímeros (m³)'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(19)
df4['Injeção de Outros Fluidos (m³)'] = df4['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'].str.split(',').str.get(20)



In [ ]:
display(df1)

,Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),...,Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)
0,2016,01/2016,Alagoas,Alagoas,PARU,4-ALS-39-AL,Mar,Estação de Paru,0.00000,1264.580,...,5505.09958,217.14500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,01/2016,Bahia,Camamu,MANATI,7-MNT-1-BAS,Mar,PLATAFORMA DE MANATI 1,0.00000,317.599,...,27140.35139,59.69300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,01/2016,Bahia,Camamu,MANATI,7-MNT-2-BAS,Mar,PLATAFORMA DE MANATI 1,0.00000,345.265,...,29504.61803,64.89700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,01/2016,Bahia,Camamu,MANATI,7-MNT-3-BAS,Mar,PLATAFORMA DE MANATI 1,0.00000,317.599,...,27140.35139,59.69300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,01/2016,Bahia,Camamu,MANATI,7-MNT-4-BAS,Mar,PLATAFORMA DE MANATI 1,0.00000,346.794,...,29644.72930,59.27600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56332,2018,12/2018,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,8-SPH-24D-SPS,Mar,FPSO CIDADE DE SÃO PAULO,NaN,NaN,...,NaN,NaN,0.00000,2331.90006,0.0,0.00000,0.0,0.0,0.0,0.0
56333,2018,12/2018,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,8-SPH-9-SPS,Mar,FPSO CIDADE DE ILHA BELA,NaN,NaN,...,NaN,NaN,1593.00734,0.00000,0.0,670.53673,0.0,0.0,0.0,0.0
56334,2018,12/2018,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,9-BRSA-1037-SPS,Mar,FPSO CIDADE DE ILHA BELA,NaN,NaN,...,NaN,NaN,0.00000,3600.45721,0.0,0.00000,0.0,0.0,0.0,0.0
56335,2018,12/2018,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,9-BRSA-1043-SPS,Mar,FPSO CIDADE DE SÃO PAULO,NaN,NaN,...,NaN,NaN,1024.93086,0.00000,0.0,375.71963,0.0,0.0,0.0,0.0


In [ ]:
df1.rename(columns={'Mês/Ano': 'Mês'}, inplace= True)

In [ ]:
df1["Mês"] = df1["Mês"].str[:2]
df2["Mês"] = df1["Mês"].str[:2]
df3["Mês"] = df1["Mês"].str[:2]
df4["Mês"] = df1["Mês"].str[:2]

In [ ]:
display(df2)

,"Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)",Ano,Mês,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),...,Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)
0,"2019,01/2019,Alagoas,Alagoas,PARU,4-ALS-39-AL,...",2019,01,Alagoas,Alagoas,PARU,4-ALS-39-AL,Mar,Estação de Paru,0,...,0,"""5557","15311""","""334","501""",,,,,
1,"2019,01/2019,Bahia,Camamu,MANATI,7-MNT-1-BAS,M...",2019,01,Bahia,Camamu,MANATI,7-MNT-1-BAS,Mar,PLATAFORMA DE MANATI 1,0,...,0,"""4893","06866""","""18","104""",,,,,
2,"2019,01/2019,Bahia,Camamu,MANATI,7-MNT-2-BAS,M...",2019,01,Bahia,Camamu,MANATI,7-MNT-2-BAS,Mar,PLATAFORMA DE MANATI 1,0,...,0,"""16796","2131""","""63","829""",,,,,
3,"2019,01/2019,Bahia,Camamu,MANATI,7-MNT-3-BAS,M...",2019,01,Bahia,Camamu,MANATI,7-MNT-3-BAS,Mar,PLATAFORMA DE MANATI 1,0,...,0,"""10433","90984""","""42","779""",,,,,
4,"2019,01/2019,Bahia,Camamu,MANATI,7-MNT-4-BAS,M...",2019,01,Bahia,Camamu,MANATI,7-MNT-4-BAS,Mar,PLATAFORMA DE MANATI 1,0,...,0,"""6544","07546""","""28","795""",,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22168,"2019,12/2019,São Paulo,Santos,SUDOESTE DE SAPI...",2019,05,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,8-SPH-24D-SPS,Mar,FPSO CIDADE DE SÃO PAULO,,...,,,0,"""3205","51864""",0,0,0,0,0
22169,"2019,12/2019,São Paulo,Santos,SUDOESTE DE SAPI...",2019,05,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,8-SPH-9-SPS,Mar,FPSO CIDADE DE ILHA BELA,,...,,,"""926","6087""",0,0,"""321","7993""",0,0
22170,"2019,12/2019,São Paulo,Santos,SUDOESTE DE SAPI...",2019,05,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,9-BRSA-1037-SPS,Mar,FPSO CIDADE DE ILHA BELA,,...,,,0,"""2696","38084""",0,0,0,0,0
22171,"2019,12/2019,São Paulo,Santos,SUDOESTE DE SAPI...",2019,05,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,9-BRSA-1043-SPS,Mar,FPSO CIDADE DE SÃO PAULO,,...,,,"""1324","30196""",0,0,"""451","18945""",0,0


In [ ]:
df2.drop(['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'], axis=1, inplace=True)


In [ ]:
df3.drop(['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'], axis=1, inplace=True)

In [ ]:
df4.drop(['Ano,Mês/Ano,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),Produção de Gás Associado (Mm³),Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)'], axis=1, inplace=True)

In [ ]:
df1.fillna(0, inplace=True)
df2.fillna(0, inplace=True)
df3.fillna(0, inplace=True)
df4.fillna(0, inplace=True)
df5.fillna(0, inplace=True)

In [ ]:
df5.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df1.reset_index(drop=True)
df2.reset_index(drop=True)
df3.reset_index(drop=True)
df4.reset_index(drop=True)
df5.reset_index(drop=True)

,Ano,Mês,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),...,Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)
0,2016,1,Alagoas,Alagoas,PARU,4-ALS-39-AL,Mar,Estação de Paru,0.0,0.0,...,0.0,0.0,0.0,0.00000,0,0.00000,0,0,0.0,0
1,2016,1,Bahia,Camamu,MANATI,7-MNT-1-BAS,Mar,PLATAFORMA DE MANATI 1,0.0,0.0,...,0.0,0.0,0.0,0.00000,0,0.00000,0,0,0.0,0
2,2016,1,Bahia,Camamu,MANATI,7-MNT-2-BAS,Mar,PLATAFORMA DE MANATI 1,0.0,0.0,...,0.0,0.0,0.0,0.00000,0,0.00000,0,0,0.0,0
3,2016,1,Bahia,Camamu,MANATI,7-MNT-3-BAS,Mar,PLATAFORMA DE MANATI 1,0.0,0.0,...,0.0,0.0,0.0,0.00000,0,0.00000,0,0,0.0,0
4,2016,1,Bahia,Camamu,MANATI,7-MNT-4-BAS,Mar,PLATAFORMA DE MANATI 1,0.0,0.0,...,0.0,0.0,0.0,0.00000,0,0.00000,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56332,2018,12,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,8-SPH-24D-SPS,Mar,FPSO CIDADE DE SÃO PAULO,0.0,0.0,...,0.0,0.0,0.0,2331.90006,0,0.00000,0,0,0.0,0
56333,2018,12,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,8-SPH-9-SPS,Mar,FPSO CIDADE DE ILHA BELA,0.0,0.0,...,0.0,0.0,0.0,0.00000,0,670.53673,0,0,0.0,0
56334,2018,12,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,9-BRSA-1037-SPS,Mar,FPSO CIDADE DE ILHA BELA,0.0,0.0,...,0.0,0.0,0.0,3600.45721,0,0.00000,0,0,0.0,0
56335,2018,12,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,9-BRSA-1043-SPS,Mar,FPSO CIDADE DE SÃO PAULO,0.0,0.0,...,0.0,0.0,0.0,0.00000,0,375.71963,0,0,0.0,0


In [ ]:
df_petroleo = pd.concat([df1, df2, df3, df4, df5])

In [ ]:
display(df_petroleo)

,Ano,Mês,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),...,Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)
0,2016,01,Alagoas,Alagoas,PARU,4-ALS-39-AL,Mar,Estação de Paru,0.0,1264.58,...,5505.09958,217.145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016,01,Bahia,Camamu,MANATI,7-MNT-1-BAS,Mar,PLATAFORMA DE MANATI 1,0.0,317.599,...,27140.35139,59.693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016,01,Bahia,Camamu,MANATI,7-MNT-2-BAS,Mar,PLATAFORMA DE MANATI 1,0.0,345.265,...,29504.61803,64.897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016,01,Bahia,Camamu,MANATI,7-MNT-3-BAS,Mar,PLATAFORMA DE MANATI 1,0.0,317.599,...,27140.35139,59.693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016,01,Bahia,Camamu,MANATI,7-MNT-4-BAS,Mar,PLATAFORMA DE MANATI 1,0.0,346.794,...,29644.7293,59.276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56332,2018,12,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,8-SPH-24D-SPS,Mar,FPSO CIDADE DE SÃO PAULO,0.0,0.0,...,0.0,0.0,0.0,2331.90006,0,0.0,0,0,0.0,0
56333,2018,12,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,8-SPH-9-SPS,Mar,FPSO CIDADE DE ILHA BELA,0.0,0.0,...,0.0,0.0,0.0,0.0,0,670.53673,0,0,0.0,0
56334,2018,12,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,9-BRSA-1037-SPS,Mar,FPSO CIDADE DE ILHA BELA,0.0,0.0,...,0.0,0.0,0.0,3600.45721,0,0.0,0,0,0.0,0
56335,2018,12,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,9-BRSA-1043-SPS,Mar,FPSO CIDADE DE SÃO PAULO,0.0,0.0,...,0.0,0.0,0.0,0.0,0,375.71963,0,0,0.0,0


In [ ]:
df_petroleo.to_csv('unificado_e_tratado.csv')

In [ ]:
print(df_petroleo.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177811 entries, 0 to 56336
Data columns (total 21 columns):
 #   Column                                            Non-Null Count   Dtype 
---  ------                                            --------------   ----- 
 0   Ano                                               177811 non-null  object
 1   Mês                                               177811 non-null  object
 2   Estado                                            177811 non-null  object
 3   Bacia                                             177811 non-null  object
 4   Campo                                             177811 non-null  object
 5   Poço                                              177811 non-null  object
 6   Ambiente                                          177811 non-null  object
 7   Instalação                                        177811 non-null  object
 8   Produção de Óleo (m³)                             177811 non-null  object
 9   Produção de Cond

In [ ]:
display(df_petroleo)

,Ano,Mês,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),...,Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)
0,2016,01,Alagoas,Alagoas,PARU,4-ALS-39-AL,Mar,Estação de Paru,0.0,1264.58,...,5505.09958,217.145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016,01,Bahia,Camamu,MANATI,7-MNT-1-BAS,Mar,PLATAFORMA DE MANATI 1,0.0,317.599,...,27140.35139,59.693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016,01,Bahia,Camamu,MANATI,7-MNT-2-BAS,Mar,PLATAFORMA DE MANATI 1,0.0,345.265,...,29504.61803,64.897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016,01,Bahia,Camamu,MANATI,7-MNT-3-BAS,Mar,PLATAFORMA DE MANATI 1,0.0,317.599,...,27140.35139,59.693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016,01,Bahia,Camamu,MANATI,7-MNT-4-BAS,Mar,PLATAFORMA DE MANATI 1,0.0,346.794,...,29644.7293,59.276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56332,2018,12,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,8-SPH-24D-SPS,Mar,FPSO CIDADE DE SÃO PAULO,0.0,0.0,...,0.0,0.0,0.0,2331.90006,0,0.0,0,0,0.0,0
56333,2018,12,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,8-SPH-9-SPS,Mar,FPSO CIDADE DE ILHA BELA,0.0,0.0,...,0.0,0.0,0.0,0.0,0,670.53673,0,0,0.0,0
56334,2018,12,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,9-BRSA-1037-SPS,Mar,FPSO CIDADE DE ILHA BELA,0.0,0.0,...,0.0,0.0,0.0,3600.45721,0,0.0,0,0,0.0,0
56335,2018,12,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,9-BRSA-1043-SPS,Mar,FPSO CIDADE DE SÃO PAULO,0.0,0.0,...,0.0,0.0,0.0,0.0,0,375.71963,0,0,0.0,0


Começando o Pipeline


In [ ]:
pip install apache-beam[interactive]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import apache_beam as beam


In [ ]:
pipe = beam.Pipeline()

datase_petroleo = (
    
    pipe
    |'Leitura do datase'>> beam.io.ReadFromText('/content/unificado_e_tratado.csv', skip_header_lines=1)
    |'Separar por virgula'>> beam.Map(lambda record: record.split(','))
    |'Filtrar estado'>> beam.Filter(lambda record: record[2] == 'Bahia')
    # |'Filtrar Produção de òleo'>> beam.Filter(lambda record: int(record[8]) > 0)
    |'Exibir o resultado'>> beam.Map(print)
)
pipe.run()

In [ ]:
display(df_petroleo)

,Ano,Mês,Estado,Bacia,Campo,Poço,Ambiente,Instalação,Produção de Óleo (m³),Produção de Condensado (m³),...,Produção de Gás Não Associado (Mm³),Produção de Água (m³),Injeção de Gás (Mm³),Injeção de Água para Recuperação Secundária (m³),Injeção de Água para Descarte (m³),Injeção de Gás Carbônico (Mm³),Injeção de Nitrogênio (Mm³),Injeção de Vapor de Água (t),Injeção de Polímeros (m³),Injeção de Outros Fluidos (m³)
0,2016,01,Alagoas,Alagoas,PARU,4-ALS-39-AL,Mar,Estação de Paru,0.0,1264.58,...,5505.09958,217.145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016,01,Bahia,Camamu,MANATI,7-MNT-1-BAS,Mar,PLATAFORMA DE MANATI 1,0.0,317.599,...,27140.35139,59.693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016,01,Bahia,Camamu,MANATI,7-MNT-2-BAS,Mar,PLATAFORMA DE MANATI 1,0.0,345.265,...,29504.61803,64.897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016,01,Bahia,Camamu,MANATI,7-MNT-3-BAS,Mar,PLATAFORMA DE MANATI 1,0.0,317.599,...,27140.35139,59.693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016,01,Bahia,Camamu,MANATI,7-MNT-4-BAS,Mar,PLATAFORMA DE MANATI 1,0.0,346.794,...,29644.7293,59.276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56332,2018,12,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,8-SPH-24D-SPS,Mar,FPSO CIDADE DE SÃO PAULO,0.0,0.0,...,0.0,0.0,0.0,2331.90006,0,0.0,0,0,0.0,0
56333,2018,12,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,8-SPH-9-SPS,Mar,FPSO CIDADE DE ILHA BELA,0.0,0.0,...,0.0,0.0,0.0,0.0,0,670.53673,0,0,0.0,0
56334,2018,12,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,9-BRSA-1037-SPS,Mar,FPSO CIDADE DE ILHA BELA,0.0,0.0,...,0.0,0.0,0.0,3600.45721,0,0.0,0,0,0.0,0
56335,2018,12,São Paulo,Santos,SUDOESTE DE SAPINHOÁ,9-BRSA-1043-SPS,Mar,FPSO CIDADE DE SÃO PAULO,0.0,0.0,...,0.0,0.0,0.0,0.0,0,375.71963,0,0,0.0,0


In [ ]:
print(df_petroleo.info())